# Module 2: Memory & State

In this module, we will learn how to add **memory** to our chains. 
By default, LLMs are **stateless** — they don't remember previous interactions. To build a chatbot, we need to manage the conversation history.

We will cover:
1.  **ChatPromptTemplate**: Structured prompts for chat.
2.  **MessagesPlaceholder**: A slot to insert chat history.
3.  **RunnableWithMessageHistory**: The LCEL way to manage state.

In [1]:
# Setup
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

load_dotenv()

llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.7)

## 1. ChatPrompts & History Slot
Instead of a single string, chat models take a list of messages (System, Human, AI).
We need a place to store the growing list of past messages. We use `MessagesPlaceholder` for this.

In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),  # This is where memory goes
        ("human", "{input}"),
    ]
)

chain = prompt | llm

## 2. Managing History (The 'Store')
We need a way to store chat history for different sessions. For this tutorial, we'll use a simple in-memory dictionary.

In [3]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Dictionary to store history for each session_id
store = {}

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

## 3. Creating the Stateful Chain
Now we wrap our basic chain with `RunnableWithMessageHistory`. This automatically handles:
1.  Fetching history from the store using `session_id`.
2.  Injecting it into the `history` placeholder.
3.  Updating the store with the new Human and AI messages.

In [4]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

## 4. Testing the Chatbot
Let's have a conversation! We must pass a `session_id` in the `config`.

In [5]:
config = {"configurable": {"session_id": "user_1"}}

response1 = with_message_history.invoke(
    {"input": "Hi, my name is Lucifer."}, 
    config=config
)
print("AI:", response1.content)

AI: Nice to meet you, Lucifer. That's an... interesting name. Are you referring to the fallen angel from mythology and literature, or perhaps it's just a coincidence? Either way, I'm here to help you with any questions or topics you'd like to discuss. What brings you here today?


In [6]:
# Follow up question - does it remember my name?
response2 = with_message_history.invoke(
    {"input": "What is my name?"}, 
    config=config
)
print("AI:", response2.content)

AI: Your name is Lucifer.


### Different Session
If we change the `session_id`, it shouldn't remember Alice.

In [7]:
config2 = {"configurable": {"session_id": "user_2"}}

response3 = with_message_history.invoke(
    {"input": "What is my name?"}, 
    config=config2
)
print("AI (User 2):", response3.content)

AI (User 2): I don't have any information about your name. This is the beginning of our conversation, and I don't retain any data about individual users. If you'd like to share your name, I'm happy to use it in our conversation.


## Exercise
Create a chatbot that pretends to be a pirate.
1.  Modify the `SystemMessage` in the prompt.
2.  Chat with it and verify it remembers context.

In [ ]:
# Your pirate bot code here
